In [1]:
import IJulia

# The julia kernel has built in support for Revise.jl, so this is the 
# recommended approach for long-running sessions:
# https://github.com/JuliaLang/IJulia.jl/blob/9b10fa9b879574bbf720f5285029e07758e50a5e/src/kernel.jl#L46-L51

# Users should enable revise within .julia/config/startup_ijulia.jl:
# https://timholy.github.io/Revise.jl/stable/config/#Using-Revise-automatically-within-Jupyter/IJulia-1

# clear console history
IJulia.clear_history()

fig_width = 7
fig_height = 5
fig_format = :retina
fig_dpi = 96

# no retina format type, use svg for high quality type/marks
if fig_format == :retina
  fig_format = :svg
elseif fig_format == :pdf
  fig_dpi = 96
  # Enable PDF support for IJulia
  IJulia.register_mime(MIME("application/pdf"))
end

# convert inches to pixels
fig_width = fig_width * fig_dpi
fig_height = fig_height * fig_dpi

# Intialize Plots w/ default fig width/height
try
  import Plots

  # Plots.jl doesn't support PDF output for versions < 1.28.1
  # so use png (if the DPI remains the default of 300 then set to 96)
  if (Plots._current_plots_version < v"1.28.1") & (fig_format == :pdf)
    Plots.gr(size=(fig_width, fig_height), fmt = :png, dpi = fig_dpi)
  else
    Plots.gr(size=(fig_width, fig_height), fmt = fig_format, dpi = fig_dpi)
  end
catch e
  # @warn "Plots init" exception=(e, catch_backtrace())
end

# Initialize CairoMakie with default fig width/height
try
  import CairoMakie
  
  CairoMakie.activate!(type = string(fig_format))
  CairoMakie.update_theme!(resolution=(fig_width, fig_height))
catch e
    # @warn "CairoMakie init" exception=(e, catch_backtrace())
end
  
# Set run_path if specified
try
  run_path = raw"/home/alexandersmolka/exess/doc/manuals/2d_3d_landing_position_comparison"
  if !isempty(run_path)
    cd(run_path)
  end
catch e
  @warn "Run path init:" exception=(e, catch_backtrace())
end


# emulate old Pkg.installed beahvior, see
# https://discourse.julialang.org/t/how-to-use-pkg-dependencies-instead-of-pkg-installed/36416/9
import Pkg
function isinstalled(pkg::String)
  any(x -> x.name == pkg && x.is_direct_dep, values(Pkg.dependencies()))
end

# ojs_define
if isinstalled("JSON") && isinstalled("DataFrames")
  import JSON, DataFrames
  global function ojs_define(; kwargs...)
    convert(x) = x
    convert(x::DataFrames.AbstractDataFrame) = Tables.rows(x)
    content = Dict("contents" => [Dict("name" => k, "value" => convert(v)) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
elseif isinstalled("JSON")
  import JSON
  global function ojs_define(; kwargs...)
    content = Dict("contents" => [Dict("name" => k, "value" => v) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
else
  global function ojs_define(; kwargs...)
    @warn "JSON package not available. Please install the JSON.jl package to use ojs_define."
  end
end


# don't return kernel dependencies (b/c Revise should take care of dependencies)
nothing


In [2]:
#| echo: false
#| output: false
path_to_exess = joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl")

"/home/alexandersmolka/exess/doc/manuals/2d_3d_landing_position_comparison/../../../exess.jl/src/ExESS.jl"

In [3]:
versioninfo()

Julia Version 1.9.2
Commit e4ee485e909 (2023-07-05 09:39 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 8 × Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-14.0.6 (ORCJIT, skylake)
  Threads: 2 on 8 virtual cores


In [4]:
#| output: false
include(path_to_exess)
using .ExESS
using LinearAlgebra

In [5]:
#| output: false
x0 = GlobalSphericalPosition(LUNAR_RADIUS, 0, 0)
v0 = LocalCartesianVelocity(250.0, 100.0, 100.0)

LocalCartesianVelocity{Float64}(250.0, 100.0, 100.0)

In [6]:
x_landing_2d = landing_position(x0, v0)

GlobalSphericalPosition{Float64}(1.7374e6, 0.018438792639089627, 0.007374965418890142)

In [7]:
traj = trajectory(x0, v0, ddx_lunar_gravity)
x_landing_3d = GlobalSphericalPosition(GlobalCartesianPosition(traj[end][4:6]))

GlobalSphericalPosition{Float64}(1.7374e6, 0.018440245307911555, 0.007375546356034765)

In [8]:
#| output: false
function compare_2d_3d(vel, elev; N=10)
    Nv, Ne = length(vel), length(elev)
    e = zeros(Nv, Ne)
    for i in eachindex(vel), j in eachindex(elev)        
        for _ in 1:N
            lon, lat = rand(2) .* [2pi, pi] .- [pi, pi/2]
            x0 = GlobalCartesianPosition(LUNAR_RADIUS .* [
                    cos(lon)*cos(lat), 
                    sin(lon)*cos(lat), 
                    sin(lat)])

            az = rand()*2pi
            v0 = GlobalCartesianVelocity(x0, LocalCartesianVelocity(vel[i] .* [
                    cos(az) * cos(elev[j]), 
                    sin(az) * cos(elev[j]), 
                    sin(elev[j])]))

            x_landing_2d = GlobalCartesianPosition(landing_position(x0, v0))
            traj = trajectory(x0, v0, ddx_lunar_gravity; reltol=1e-6)
            x_landing_3d = GlobalCartesianPosition(traj[end][4:6])

            e[i,j] += norm(vec(x_landing_2d) - vec(x_landing_3d)) / N / LUNAR_RADIUS
        end
    end
    return e
end

compare_2d_3d (generic function with 1 method)

In [9]:
vel = 250:250:2000
elev = deg2rad.(10:10:80)
e = compare_2d_3d(vel, elev)

8×8 Matrix{Float64}:
 3.2675e-6   1.58987e-6  1.00849e-6  …  3.43475e-7  2.17701e-7  1.05911e-7
 3.28522e-6  1.62074e-6  1.04814e-6     3.78796e-7  2.44216e-7  1.16128e-7
 3.32543e-6  1.69164e-6  1.12286e-6     4.35922e-7  2.81118e-7  1.46596e-7
 3.42594e-6  1.83573e-6  1.26074e-6     5.33725e-7  3.55168e-7  1.72692e-7
 3.74129e-6  2.19494e-6  1.62618e-6     6.73647e-7  3.94038e-7  1.79379e-7
 5.5685e-6   3.40078e-6  2.24533e-6  …  8.76605e-7  5.72617e-7  2.6612e-7
 8.37101e-6  4.51938e-6  2.90517e-6     1.06504e-6  6.864e-7    3.11505e-7
 2.71783e-6  2.98604e-6  2.69721e-6     1.28305e-6  8.0765e-7   3.8365e-7

In [10]:
max(e...)

8.371005582249139e-6

In [11]:
#| output: false
#| echo: false
using CairoMakie
include(joinpath(@__DIR__, "..", "..", "resources", "julia", "theme.jl"))

function plot_compare_2d_3d(v, elev, e)
    fig = Figure(; resolution=(600,400))
    ax = Axis(fig[1,1];
        xlabel="Velocity [m/s]",
        ylabel="Elevation Angle [rad]")

    hm = heatmap!(ax, v, rad2deg.(elev), e .* 1e5, colormap=lipari)
    Colorbar(fig[1,2], hm, label="Absolute Error, [10⁻⁵ Lunar Radius]")
    save(joinpath(@__DIR__, "compare_2d_3d.pdf"), fig, pt_per_unit=4)
    save(joinpath(@__DIR__, "compare_2d_3d.svg"), fig, pt_per_unit=4)
    save(joinpath(@__DIR__, "compare_2d_3d.png"), fig, px_per_unit=4)

    return nothing
end

vel = 50:50:2000
elev = deg2rad.(2.5:2.5:85)
e = compare_2d_3d(vel, elev; N=100)
plot_compare_2d_3d(vel, elev, e)